# 寫成def參數一次爬取

In [1]:
#依順序import進module
import sys
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re


def craw_all(i): 
    sys.stdout.write('正在抓取捐款進度報告第'+ '%s'%i +'頁')
    craw = requests.get('https://tw.appledaily.com/charity/projlist/Page/%s'%i)
    soup = BeautifulSoup(craw.text, 'lxml')
    #抓取220個網頁中每一張捐款進度報告表
    all_cells = soup.find_all(re.compile("td"))
    all_cells[:6] 
    list = []
    for s in all_cells[6:]:   
        data = s.text
        list.append(data)
    matrix=[]                        #或可先轉為ndarry再reshape  np.array(list).reshape
    for m in range(0,len(list),6):
        matrix.append(list[m:m+6])   
    matrix=np.array(matrix) 
    matrixT=matrix.T
    tables=pd.DataFrame()  
    for t in range(0,len(matrixT)):
        tables['%s'%t]=matrixT[t]
    #存成表格
    tables = tables.rename(columns={'0':'編號','1':'報導標題','2':'刊登日期','3':'狀態','4':'累計(元)','5':'捐款明細'})
    #加入url位址
    article_url = [tag.find("a")['href'] for tag in soup.find_all("h2")[3:]]  
    tables['text_url'] = pd.Series(article_url)  
    web_url = 'https://tw.appledaily.com/'  
    detail_url = [web_url + tag['href'] for tag in soup.find_all("a", {"class":"details"})] 
    tables['detail_url'] = pd.Series(detail_url) 
    #存檔輸出表格
    tables.to_excel("./my_crawler_all/reports/report_%s"%i+".xlsx")

    #將每一張表格中的每一則報導全文存檔輸出 或with open(filename,'w')
    for url in article_url:
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "lxml")
        main_title = soup.find("article", {"class":"ndArticle_leftColumn"}).find("h1").text   
        sub_title = soup.find("article", {"class":"ndArticle_leftColumn"}).find("h2").text 
        paragraph = [tag.text for tag in soup.find("div", {"class":"ndArticle_margin"}).find_all("p")]
        df = pd.DataFrame()
        df['text']=pd.Series(paragraph)
        df.to_csv("./my_crawler_all/articles/"+"page%s"%i+"-"+url.split('/')[-2]+"-"+url.split('/')[-1]+".csv",encoding='utf_8_sig') #bylines

In [2]:
for i in range(101, 121):
    craw_all(i)   #指定範圍, 爬取中間20頁的表格與約20*20篇全文

正在抓取捐款進度報告第101頁正在抓取捐款進度報告第102頁正在抓取捐款進度報告第103頁正在抓取捐款進度報告第104頁正在抓取捐款進度報告第105頁正在抓取捐款進度報告第106頁正在抓取捐款進度報告第107頁正在抓取捐款進度報告第108頁正在抓取捐款進度報告第109頁正在抓取捐款進度報告第110頁正在抓取捐款進度報告第111頁正在抓取捐款進度報告第112頁正在抓取捐款進度報告第113頁正在抓取捐款進度報告第114頁正在抓取捐款進度報告第115頁正在抓取捐款進度報告第116頁正在抓取捐款進度報告第117頁正在抓取捐款進度報告第118頁正在抓取捐款進度報告第119頁正在抓取捐款進度報告第120頁

In [ ]:
for i in range(2, 221):
    craw_all(i)   #全數輸出    把Wins上的檔案以linux aws3指令搬到aws平臺上,利用EC2演算法平臺將py寫入sh陸續進行文章斷詞與探勘分析